# Major Leagues

a project for EECS 731 by Benjamin Wyss

Examining fivethirtyeight soccer power index (spi) data sets to build a regression model which predicts the scores of soccer matches

###### python imports

In [14]:
import numpy as np
import pandas as pd
import sklearn as skl
import matplotlib.pyplot as plt
plt.close('all')
import warnings
warnings.filterwarnings('ignore')

### Reading Data Sets From CSV

fivethirtyeight Soccer SPI Ratings and Matches

Taken from: https://github.com/fivethirtyeight/data/tree/master/soccer-spi on 10/1/20

Only the spi_matches dataset is examined because it contains all of the match information and historical spi data needed to build the target regression model. The spi_matches_latest data set is a subset of the spi_matches dataset, and thus it can be discarded without losing any additional match data samples. The remaining data sets include information about current spi ratings, but the target regression model should perform better at predicting match scores based on ratings from when a match occured rather than from current ratings, so these data sets are discarded as well.

###### Soccer Matches and SPI Ratings Data Set

In [15]:
df = pd.read_csv('../data/raw/spi_matches.csv')

In [16]:
df

,season,date,league_id,league,team1,team2,spi1,spi2,prob1,prob2,...,importance1,importance2,score1,score2,xg1,xg2,nsxg1,nsxg2,adj_score1,adj_score2
0,2016,2016-07-09,7921,FA Women's Super League,Liverpool Women,Reading,51.56,50.42,0.4389,0.2767,...,NaN,NaN,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2016,2016-07-10,7921,FA Women's Super League,Arsenal Women,Notts County Ladies,46.61,54.03,0.3572,0.3608,...,NaN,NaN,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2016,2016-07-10,7921,FA Women's Super League,Chelsea FC Women,Birmingham City,59.85,54.64,0.4799,0.2487,...,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2016,2016-07-16,7921,FA Women's Super League,Liverpool Women,Notts County Ladies,53.00,52.35,0.4289,0.2699,...,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2016,2016-07-17,7921,FA Women's Super League,Chelsea FC Women,Arsenal Women,59.43,60.99,0.4124,0.3157,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42168,2020,2021-05-30,1871,Spanish Segunda Division,Mirandes,CD Sabadell,32.09,30.30,0.4473,0.2862,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42169,2020,2021-05-30,1871,Spanish Segunda Division,AD Alcorcon,Espanyol,33.41,62.03,0.1738,0.5834,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42170,2020,2021-05-30,1871,Spanish Segunda Division,Málaga,Castellon,35.64,31.26,0.4759,0.2216,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42171,2020,2021-05-30,1871,Spanish Segunda Division,FC Cartagena,Girona FC,29.81,39.15,0.3196,0.3964,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## The Big Ideas

Feature engineering and transformation can add value to this data set for building a regression model in the following ways:

(1): By selecting only the most promising and score-correlated attributes as input features, the target regression model will be able to best utilize correlations between features and match scores to increase overall model accuracy.

(2): By one-hot encoding league and team names, the target regression model can gain additional information about how many points specific teams score throughout specific leagues which will aid its prediction accuracy.

## Exploratory Data Analysis

### Cleaning the data sets

In [17]:
df

,season,date,league_id,league,team1,team2,spi1,spi2,prob1,prob2,...,importance1,importance2,score1,score2,xg1,xg2,nsxg1,nsxg2,adj_score1,adj_score2
0,2016,2016-07-09,7921,FA Women's Super League,Liverpool Women,Reading,51.56,50.42,0.4389,0.2767,...,NaN,NaN,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2016,2016-07-10,7921,FA Women's Super League,Arsenal Women,Notts County Ladies,46.61,54.03,0.3572,0.3608,...,NaN,NaN,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2016,2016-07-10,7921,FA Women's Super League,Chelsea FC Women,Birmingham City,59.85,54.64,0.4799,0.2487,...,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2016,2016-07-16,7921,FA Women's Super League,Liverpool Women,Notts County Ladies,53.00,52.35,0.4289,0.2699,...,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2016,2016-07-17,7921,FA Women's Super League,Chelsea FC Women,Arsenal Women,59.43,60.99,0.4124,0.3157,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42168,2020,2021-05-30,1871,Spanish Segunda Division,Mirandes,CD Sabadell,32.09,30.30,0.4473,0.2862,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42169,2020,2021-05-30,1871,Spanish Segunda Division,AD Alcorcon,Espanyol,33.41,62.03,0.1738,0.5834,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42170,2020,2021-05-30,1871,Spanish Segunda Division,Málaga,Castellon,35.64,31.26,0.4759,0.2216,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42171,2020,2021-05-30,1871,Spanish Segunda Division,FC Cartagena,Girona FC,29.81,39.15,0.3196,0.3964,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Transforming the data sets


### Visualizing the Data

First, some basic statistics of the data set and correlation coefficients are calculated

In [18]:
df.describe()

,season,league_id,spi1,spi2,prob1,prob2,probtie,proj_score1,proj_score2,importance1,importance2,score1,score2,xg1,xg2,nsxg1,nsxg2,adj_score1,adj_score2
count,42173.000000,42173.000000,42173.000000,42173.000000,42173.000000,42173.000000,42173.000000,42173.000000,42173.000000,29572.000000,29572.00000,34268.000000,34268.000000,18273.000000,18273.000000,18273.000000,18273.000000,18273.000000,18273.000000
mean,2018.411543,2181.590117,44.756930,44.720523,0.446570,0.300422,0.253008,1.516175,1.172247,31.306398,30.60534,1.523112,1.181715,1.500572,1.170989,1.407696,1.134884,1.540470,1.190836
std,1.174531,901.753292,18.958686,18.973440,0.157805,0.142912,0.047112,0.425189,0.418712,26.218610,25.85920,1.282570,1.142430,0.829435,0.735030,0.652930,0.570753,1.245991,1.128228
min,2016.000000,1818.000000,3.880000,4.040000,0.027100,0.003200,0.000000,0.250000,0.200000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2018.000000,1849.000000,30.940000,30.930000,0.346800,0.207400,0.235300,1.240000,0.910000,10.700000,10.20000,1.000000,0.000000,0.880000,0.620000,0.950000,0.730000,1.050000,0.000000
50%,2018.000000,1874.000000,42.670000,42.570000,0.437400,0.285000,0.261100,1.450000,1.120000,26.000000,25.20000,1.000000,1.000000,1.370000,1.040000,1.320000,1.050000,1.050000,1.050000
75%,2019.000000,2160.000000,58.120000,58.070000,0.535100,0.375000,0.281500,1.710000,1.380000,45.500000,44.60000,2.000000,2.000000,1.970000,1.560000,1.750000,1.430000,2.100000,2.100000
max,2020.000000,9541.000000,96.570000,96.780000,0.977500,0.899200,0.453700,4.900000,4.010000,100.000000,100.00000,11.000000,9.000000,7.070000,6.200000,6.890000,5.920000,9.150000,7.930000


In [19]:
df.corr()

,season,league_id,spi1,spi2,prob1,prob2,probtie,proj_score1,proj_score2,importance1,importance2,score1,score2,xg1,xg2,nsxg1,nsxg2,adj_score1,adj_score2
season,1.000000,-0.003090,-0.173172,-0.174024,-0.045921,0.059279,-0.026003,-0.019484,0.071384,0.005693,0.001886,-0.011913,0.015250,0.017753,0.052834,-0.015834,0.004984,-0.015833,0.014908
league_id,-0.003090,1.000000,-0.011040,-0.002200,-0.035165,0.028932,0.030021,-0.055666,0.002632,-0.113949,-0.113123,-0.027612,-0.008804,-0.042356,-0.052965,-0.031415,-0.042875,-0.059734,-0.043892
spi1,-0.173172,-0.011040,1.000000,0.725621,0.413552,-0.338577,-0.358158,0.411340,-0.285175,0.237619,0.104540,0.145003,-0.077101,0.222491,-0.073344,0.257511,-0.123095,0.153642,-0.070863
spi2,-0.174024,-0.002200,0.725621,1.000000,-0.296836,0.363344,-0.107905,-0.211839,0.348759,0.098546,0.232628,-0.080582,0.128151,-0.086743,0.215758,-0.108830,0.229480,-0.089843,0.146453
prob1,-0.045921,-0.035165,0.413552,-0.296836,1.000000,-0.955708,-0.450476,0.896470,-0.845741,0.268296,-0.211493,0.310997,-0.259674,0.411614,-0.352685,0.484448,-0.438040,0.320732,-0.264582
prob2,0.059279,0.028932,-0.338577,0.363344,-0.955708,1.000000,0.167763,-0.776506,0.932239,-0.217204,0.260338,-0.271946,0.290257,-0.363794,0.396134,-0.430814,0.485393,-0.283134,0.294766
probtie,-0.026003,0.030021,-0.358158,-0.107905,-0.450476,0.167763,1.000000,-0.647291,0.004971,-0.236662,-0.077574,-0.214805,-0.008764,-0.283652,-0.006800,-0.326007,0.011108,-0.222011,0.002025
proj_score1,-0.019484,-0.055666,0.411340,-0.211839,0.896470,-0.776506,-0.647291,1.000000,-0.556369,0.250422,-0.168499,0.329584,-0.176136,0.429640,-0.245749,0.509225,-0.313489,0.339363,-0.184422
proj_score2,0.071384,0.002632,-0.285175,0.348759,-0.845741,0.932239,0.004971,-0.556369,1.000000,-0.197060,0.232949,-0.207023,0.299402,-0.292918,0.411639,-0.353970,0.501245,-0.225094,0.304332
importance1,0.005693,-0.113949,0.237619,0.098546,0.268296,-0.217204,-0.236662,0.250422,-0.197060,1.000000,0.329041,0.067201,-0.063450,0.114217,-0.054501,0.137417,-0.074613,0.090451,-0.050411


###### Generating Interesting Plots

###### Intuitions Gained From Visualizations

### Model Construction

A variety of machine learning models will be tested and compared

In [20]:
array = df.values

### Analysis and Testing of Regression Models

### Results